# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 57 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2302.06614


extracting tarball to tmp_2302.06614...

 done.
Retrieving document from  https://arxiv.org/e-print/2302.06617


extracting tarball to tmp_2302.06617...

...

extracting tarball to tmp_2302.07256...

 done.


Retrieving document from  https://arxiv.org/e-print/2302.07258


extracting tarball to tmp_2302.07258...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07057-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07057) | **Morphology of the gas-rich debris disk around HD 121617 with SPHERE  observations in polarized light**  |
|| Clément Perrot, et al. -- incl., <mark>Johan Olofsson</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| *12 pages, 7 figures. Accepted in A&A (06/02/2023)*|
|**Abstract**| Debris disks are the signposts of collisionally eroding planetesimal circumstellar belts, whose study can put important constraints on the structure of extrasolar planetary systems. The best constraints on the morphology of disks are often obtained from spatially resolved observations in scattered light. Here, we investigate the young (~16 Myr) bright gas-rich debris disk around HD121617. We use new scattered-light observations with VLT/SPHERE to characterize the morphology and the dust properties of this disk. From these properties we can then derive constraints on the physical and dynamical environment of this system, for which significant amounts of gas have been detected. The disk morphology is constrained by linear-polarimetric observations in the J band. Based on our modeling results and archival photometry, we also model the SED to put constraints on the total dust mass and the dust size distribution. We explore different scenarios that could explain these new constraints. We present the first resolved image in scattered light of the debris disk HD121617. We fit the morphology of the disk, finding a semi-major axis of 78.3$\pm$0.2 au, an inclination of 43.1$\pm$0.2{\deg} and a position angle of the major axis with respect to north, of 239.8$\pm$0.3{\deg}, compatible with the previous continuum and CO detection with ALMA. Our analysis shows that the disk has a very sharp inner edge, possibly sculpted by a yet-undetected planet or gas drag. While less sharp, its outer edge is steeper than expected for unperturbed disks, which could also be due to a planet or gas drag, but future observations probing the system farther from the main belt would help explore this further. The SED analysis leads to a dust mass of 0.21$\pm$0.02 M$_{\oplus}$ and a minimum grain size of 0.87$\pm$0.12 $\mu$m, smaller than the blowout size by radiation pressure, which is not unexpected for very bright col... |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07234-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07234) | **JADES Imaging of GN-z11: Revealing the Morphology and Environment of a  Luminous Galaxy 430 Myr After the Big Bang**  |
|| Sandro Tacchella, et al. -- incl., <mark>Jakob M. Helton</mark>, <mark>Hans-Walter Rix</mark>, <mark>Anna de Graaff</mark>, <mark>Lily Whitler</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| *Submitted to ApJ; 18 pages, 8 figures; comments welcome*|
|**Abstract**| We present JWST NIRCam 9-band near-infrared imaging of the luminous $z=10.6$ galaxy GN-z11 from the JWST Advanced Deep Extragalactic Survey (JADES) of the GOODS-N field. We find a spectral energy distribution (SED) entirely consistent with the expected form of a high-redshift galaxy: a clear blue continuum from 1.5 to 4 microns with a complete dropout in F115W. The core of GN-z11 is extremely compact in JWST imaging. We analyze the image with a two-component model, using a point source and a S\'{e}rsic profile that fits to a half-light radius of 200 pc and an index $n=0.9$. We find a low-surface brightness haze about $0.4''$ to the northeast of the galaxy, which is most likely a foreground object but might be a more extended component of GN-z11. At a spectroscopic redshift of 10.60 (Bunker et al. 2023), the comparison of the NIRCam F410M and F444W images spans the Balmer jump. From population synthesis modeling, here assuming no light from an active galactic nucleus, we reproduce the SED of GN-z11, finding a stellar mass of $\sim$$10^{9}~M_{\odot}$, a star-formation rate of $\sim$$20~M_{\odot}~\mathrm{yr}^{-1}$ and a young stellar age of $\sim$$20~\mathrm{Myr}$. As massive galaxies at high redshift are likely to be highly clustered, we search for faint neighbors of GN-z11, finding 9 galaxies out to $\sim$5 comoving Mpc transverse with photometric redshifts consistent with $z=10.6$, and a 10$^{\rm th}$ more tentative dropout only $3''$ away. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07256-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07256) | **JADES NIRSpec Spectroscopy of GN-z11: Lyman-$α$ emission and  possible enhanced nitrogen abundance in a $z=10.60$ luminous galaxy**  |
|| Andrew J. Bunker, et al. -- incl., <mark>Anna de Graaff</mark>, <mark>Hans-Walter Rix</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| *Submitted to Astronomy & Astrophysics, 14 pages, 9 figures*|
|**Abstract**| We present JADES JWST/NIRSpec spectroscopy of GN-z11, the most luminous candidate $z>10$ Lyman break galaxy in the GOODS-North field with $M_{UV}=-21.5$. We derive a redshift of $z=10.603$ (lower than previous determinations) based on multiple emission lines in our low and medium resolution spectra over $0.8-5.3\,\mu$m. We significantly detect the continuum and measure a blue rest-UV spectral slope of $\beta=-2.4$. Remarkably, we see spatially-extended Lyman-$\alpha$ in emission (despite the highly-neutral IGM expected at this early epoch), offset 555 km/s redward of the systemic redshift. From our measurements of collisionally-excited lines of both low- and high-ionization (including [O II] $\lambda3727$, [Ne III] $\lambda 3869$ and C III] $\lambda1909$) we infer a high ionization parameter ($\log U\sim -2$). We detect the rarely-seen N IV] $\lambda1486$ and N III]$\lambda1748$ lines in both our low and medium resolution spectra, with other high ionization lines seen in low resolution spectrum such as He II (blended with O III]) and C IV (with a possible P-Cygni profile). Based on the observed rest-UV line ratios, we cannot conclusively rule out photoionization from AGN. The high C III]/He II ratios, however, suggest a likely star-formation explanation. If the observed emission lines are powered by star formation, then the strong N III] $\lambda1748$ observed may imply an unusually high $N/O$ abundance. Balmer emission lines (H$\gamma$, H$\delta$) are also detected, and if powered by star formation rather than an AGN we infer a star formation rate of $\sim 20-30 M_{\odot}\,\rm yr^{-1}$ (depending on the IMF) and low dust attenuation. Our NIRSpec spectroscopy confirms that GN-z11 is a remarkable galaxy with extreme properties seen 430 Myr after the Big Bang. |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07034-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07034) | **The foreground transfer function for HI intensity mapping signal  reconstruction: MeerKLASS and precision cosmology applications**  |
|| Steven Cunnington, et al. -- incl., <mark>Yichao Li</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| *25 pages, 20 figures. See Figure 4 for the main demonstration of the transfer function's performance for reconstructing signal loss from foreground cleaning. Submitted to MNRAS for publication*|
|**Abstract**| Blind cleaning methods are currently the preferred strategy for handling foreground contamination in single-dish HI intensity mapping surveys. Despite the increasing sophistication of blind techniques, some signal loss will be inevitable across all scales. Constructing a corrective transfer function using mock signal injection into the contaminated data has been a practice relied on for HI intensity mapping experiments. However, assessing whether this approach is viable for future intensity mapping surveys where precision cosmology is the aim, remains unexplored. In this work, using simulations, we validate for the first time the use of a foreground transfer function to reconstruct power spectra of foreground-cleaned low-redshift intensity maps and look to expose any limitations. We reveal that even when aggressive foreground cleaning is required, which causes ${>}\,50\%$ negative bias on the largest scales, the power spectrum can be reconstructed using a transfer function to within sub-percent accuracy. We specifically outline the recipe for constructing an unbiased transfer function, highlighting the pitfalls if one deviates from this recipe, and also correctly identify how a transfer function should be applied in an auto-correlation power spectrum. We validate a method that utilises the transfer function variance for error estimation in foreground-cleaned power spectra. Finally, we demonstrate how incorrect fiducial parameter assumptions (up to ${\pm}100\%$ bias) in the generation of mocks, used in the construction of the transfer function, do not significantly bias signal reconstruction or parameter inference (inducing ${<}\,5\%$ bias in recovered values). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07043-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07043) | **Computation of stochastic background from extreme mass ratio inspiral  populations for LISA**  |
|| Federico Pozzoli, et al. -- incl., <mark>Matteo Bonetti</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| **|
|**Abstract**| Extreme mass ratio inspirals (EMRIs) are among the primary targets for the Laser Interferometer Space Antenna (LISA). The extreme mass ratios of these systems result in relatively weak GW signals, that can be individually resolved only for cosmologically nearby sources (up to $z\approx2$). The incoherent piling up of the signal emitted by unresolved EMRIs generate a confusion noise, that can be formally treated as a stochastic GW background (GWB). In this paper, we estimate the level of this background considering a collection of astrophysically motivated EMRI models, spanning the range of uncertainties affecting EMRI formation. To this end, we employed the innovative Augmented Analytic Kludge waveforms and used the full LISA response function. For each model, we compute the GWB SNR and the number of resolvable sources. Compared to simplified computations of the EMRI signals from the literature, we find that for a given model the GWB SNR is lower by a factor of $\approx 2$ whereas the number of resolvable sources drops by a factor 3-to-5. Nonetheless, the vast majority of the models result in potentially detectable GWB which can also significantly contribute to the overall LISA noise budget in the 1-10 mHz frequency range. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.07258-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.07258) | **Accelerations of stars in central 2-7 arcsec from Sgr A***  |
|| A. Young, et al. -- incl., <mark>J. Stadler</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| *Accepted for publication in Astronomy and Astrophysics, Volume 670*|
|**Abstract**| This work presents the results from extending the long-term monitoring program of stellar motions within the Galactic Center to include stars with separations of 2 - 7 arcseconds from the compact radio source, Sgr A*. In comparison to the well studied inner 2 arcsec, a longer baseline in time is required to study these stars. With 17 years of data, a sufficient number of positions along the orbits of these outer stars can now be measured. This was achieved by designing a source finder to track the positions of ~ 2000 stars in NACO/VLT adaptive-optics-assisted images of the Galactic Center from 2002 to 2019. Of the studied stars, 54 exhibit significant accelerations toward Sgr A*, most of which have separations of between 2 and 3 arcseconds from the black hole. A further 20 of these stars have measurable radial velocities from SINFONI/VLT stellar spectra, which allows for the calculation of the orbital elements for these stars, thus increasing the number of known orbits in the Galactic Center by ~ 40 %. With orbits, we can consider which structural features within the Galactic Center nuclear star cluster these stars belong to. Most of the stars have orbital solutions that are consistent with the known clockwise rotating disk feature. Further, by employing Monte Carlo sampling for stars without radial velocity measurements, we show that many stars have a subset of possible orbits that are consistent with one of the known disk features within the Galactic Center. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.06614-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.06614) | **In-situ detection of Europa's water plumes is harder than previously  thought**  |
|| Rowan Dayton-Oxland, et al. -- incl., <mark>Arnaud Maheiux</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| **|
|**Abstract**| Europa's subsurface ocean is a potential candidate for life in the outer solar system. It is thought that plumes may exist which eject ocean material out into space, which may be detected by a spacecraft flyby. Previous work on the feasibility of these detections has assumed a collisionless model of the plume particles. New models of the plumes including particle collisions have shown that a shock can develop in the plume interior as rising particles collide with particles falling back to the moon's surface, limiting the plume's altitude. Results show that the region over Europa's surface within which plumes would be separable from the H$_2$O atmosphere by JUICE (the region of separability) is reduced by up to a half with the collisional model compared to the collisionless model. Putative plume sources which are on the border of the region of separability for the collisionless model cannot be separated from the atmosphere when the shock is considered for a mass flux case of 100kg/s. Increasing the flyby altitude by 100km such that the spacecraft passes above the shock canopy results in a reduction in region of separability by a third, whilst decreasing the flyby altitude by 100km increases the region of separability by the same amount. We recommend flybys pass through or as close to the shock as possible to sample the most high-density region. If the spacecraft flies close to the shock, the structure of the plume could be resolvable using the neutral mass spectrometer on JUICE, allowing us to test models of the plume physics and understand the underlying physics of Europa's plumes. As the altitude of the shock is uncertain and dependent on unpredictable plume parameters, we recommend flybys be lowered where possible to reduce the risk of passing above the shock and losing detection coverage, density and duration. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2302.06825-b31b1b.svg)](https://arxiv.org/abs/arXiv:2302.06825) | **Multi-instrument Comparative Study of Temperature, Number Density, and  Emission Measure during the Precursor Phase of a Solar Flare**  |
|| <mark>Nian Liu</mark>, Ju Jing, Yan Xu, Haimin Wang |
|*Appeared on*| *2023-02-15*|
|*Comments*| *10 pages, 7 figures*|
|**Abstract**| We present a multi-instrument study of the two precursor brightenings prior to the M6.5 flare (SOL2015-06-22T18:23) in the NOAA Active Region 12371, with a focus on the temperature (T), electron number density (n), and emission measure (EM). The data used in this study were obtained from four instruments with a variety of wavelengths, i.e., the Solar Dynamics Observatory's Atmospheric Imaging Assembly (AIA), in six extreme ultraviolet (EUV) passbands; the Expanded Owens Valley Solar Array (EOVSA) in microwave (MW); the Reuven Ramaty High Energy Solar Spectroscopic Imager (RHESSI) in hard X-rays (HXR); and the Geostationary Operational Environmental Satellite (GOES) in soft X-rays (SXR). We compare the temporal variations of T, n, and EM derived from the different data sets. Here are the key results. (1) GOES SXR and AIA EUV have almost identical EM variations (1.5-3x10^48 per cm^3) and very similar T variations, from 8 to 15 million Kelvin (MK). (2) Listed from highest to lowest, EOVSA MW provides the highest temperature variations (15-60 MK), followed by RHESSI HXR (10-24 MK), then GOES SXR and AIA EUV (8-15 MK). (3) The EM variation from the RHESSI HXR measurements is always less than the values from AIA EUV and GOES SXR by at most 20 times. The number density variation from EOVSA MW is greater than the value from AIA EUV by at most 100 times. The results quantitatively describe the differences in the thermal parameters at the precursor phase, as measured by different instruments operating at different wavelength regimes and for different emission mechanisms. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2302.07057.md
    + _build/html/tmp_2302.07057/./images/test_triangle.png
    + _build/html/tmp_2302.07057/./images/Qphi-Uphi-SNRmap.png
    + _build/html/tmp_2302.07057/./images/semi-major_axis_cut.png
exported in  _build/html/2302.07234.md
    + _build/html/tmp_2302.07234/./ratio_flux_size.png
    + _build/html/tmp_2302.07234/./SED.png
    + _build/html/tmp_2302.07234/./FP_residual.png
exported in  _build/html/2302.07256.md
    + _build/html/tmp_2302.07256/./figures/all_emlines_gnz11_v2.png
    + _build/html/tmp_2302.07256/./figures/GN-z11_v3GTO_1D_2D_micron_3pix.png
    + _build/html/tmp_2302.07256/./figures/GN-z11_Lya_zoom_hist.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\mum}{\SI{\micro\meter}}$
$\newcommand{\deg}{^{\circ}}$
$\newcommand{\degb}{^{\circ}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\mum$}{\SI{\micro\meter}}$
$\newcommand{$\deg$}{^{\circ}}$
$\newcommand{$\deg$b}{^{\circ}}$</div>



<div id="title">

# Morphology of the gas-rich debris disk around HD 121617 with SPHERE observations in polarized light

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07057-b31b1b.svg)](https://arxiv.org/abs/2302.07057)<mark>Appeared on: 2023-02-15</mark> - _12 pages, 7 figures. Accepted in A&A (06/02/2023)_

</div>
<div id="authors">

Clément Perrot, et al. -- incl., <mark><mark>Johan Olofsson</mark></mark>

</div>
<div id="abstract">

**Abstract:** Debris disks are the signposts of collisionally eroding planetesimal circumstellar belts, whose study can put important constraints on the structure of extrasolar planetary systems. The best constraints on the morphology of disks are often obtained from spatially resolved observations in scattered light. Here, we investigate the young ($\sim$16 Myr) bright gas-rich debris disk around HD 121617.We use new scattered-light observations with VLT/SPHERE to characterize the morphology and the dust properties of this disk. From these properties we can then derive constraints on the physical and dynamical environment of this system, for which significant amounts of gas have been detected.The disk morphology is constrained by linear-polarimetric observations in the\textit{J}band. Based on our modeling results and archival photometry, we also model the SED to put constraints on the total dust mass and the dust size distribution. We explore different scenarios that could explain these new constraints.We present the first resolved image in scattered light of the debris disk HD 121617. We fit the morphology of the disk, finding a semi-major axis of$78.3\pm0.2$au, an inclination of$43.1\pm0.2^{\circ}$and a position angle of the major axis with respect to north, of$239.8\pm0.3^{\circ}$, compatible with the previous continuum and CO detection with ALMA. Our analysis shows that the disk has a very sharp inner edge, possibly sculpted by a yet-undetected planet or gas drag. While less sharp, its outer edge is steeper than expected for unperturbed disks, which could also be due to a planet or gas drag, but future observations probing the system farther from the main belt would help explore this further. The SED analysis leads to a dust mass of$0.21\pm0.02 M_{\oplus}$and a minimum grain size of$0.87\pm0.12 $\mum$$, smaller than the blowout size by radiation pressure, which is not unexpected for very bright collisionally active disks.

</div>

<div id="div_fig1">

<img src="tmp_2302.07057/./images/test_triangle.png" alt="Fig4" width="100%"/>

**Figure 4. -** Corner-plot for the MCMC analysis. From left to right and top to bot: $a$($r_0$), $i$, $\phi$, $g$, $\alpha_\mathrm{in}$, $\alpha_\mathrm{out}$, $e$ and $\omega$. Middle dash-lines are the more probable value and external dash-lines are the 1-$\sigma$ error. (*cornerplot*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07057/./images/Qphi-Uphi-SNRmap.png" alt="Fig5" width="100%"/>

**Figure 5. -** Left to right: $Q_\phi$ image, $U_\phi$ image, and S/N map of the $J$ band observations of HD 121617. North is up, east is left. (*observation*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07057/./images/semi-major_axis_cut.png" alt="Fig7" width="100%"/>

**Figure 7. -** $Q_{\phi}$ radial profile following the projected major axis (6 pixels width) from the South-West to the North-East of the disk with the corresponding error bars (black points). The radial profile of the best model (red dashed line) and the Noise level obtained with the azimuthal standard deviation of the $U_{\phi}$ image (blue dotted line).
 (*radialprofile*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\lya}{\ensuremath{{\rm Ly}\alpha}}$
$\newcommand{\kms}{\ensuremath{{\rm\;km\;s^{-1}}}}$
$\newcommand{\Mpc}{\ensuremath{{\rm\;Mpc}}}$
$\newcommand{\Myr}{\ensuremath{{\rm\;Myr}}}$
$\newcommand{\Msun}{\ensuremath{{\rm\;M_\odot}}}$
$\newcommand{\yr}{\ensuremath{{\rm\;yr}}}$
$\newcommand{\cm}{\ensuremath{{\rm\;cm}}}$
$\newcommand{\ergscms}{\ensuremath{{\rm\;ergs\;cm^{-2}\;s^{-1}}}}$
$\newcommand{\ergss}{\ensuremath{{\rm\;ergs\;s^{-1}}}}$
$\newcommand{\mic}{\ensuremath{\mu\rm m}}$
$\newcommand{\ciii}{C \textsc{iii}]}$
$\newcommand{\niii}{N \textsc{iii}]}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\niv}{N \textsc{iv}}$
$\newcommand{\todo}[1]{{\color{blue} \tt #1}}$
$\newcommand{\tbc}[1]{#1 ({\color{red} \tt TBC})}$
$\newcommand{\tbd}{({\color{red} \tt TBD})}$
$\newcommand{\nod}{\nodata}$
$\newcommand{\}{natexlab}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\url$}[1]{\href{#1}{#1}}$
$\newcommand{$\dodoi$}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{$\doeprint$}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{$\doarXiv$}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{$\lya$}{$\ensuremath${{\rm Ly}\alpha}}$
$\newcommand{$\kms$}{$\ensuremath${{\rm\;km\;s^{-1}}}}$
$\newcommand{$\Mpc$}{$\ensuremath${{\rm\;Mpc}}}$
$\newcommand{$\Myr$}{$\ensuremath${{\rm\;Myr}}}$
$\newcommand{$\Msun$}{$\ensuremath${{\rm\;M_\odot}}}$
$\newcommand{$\yr$}{$\ensuremath${{\rm\;yr}}}$
$\newcommand{$\cm$}{$\ensuremath${{\rm\;cm}}}$
$\newcommand{$\ergscms$}{$\ensuremath${{\rm\;ergs\;cm^{-2}\;s^{-1}}}}$
$\newcommand{$\ergss$}{$\ensuremath${{\rm\;ergs\;s^{-1}}}}$
$\newcommand{$\mic$}{$\ensuremath${\mu\rm m}}$
$\newcommand{$\ciii$}{C $\textsc${iii}]}$
$\newcommand{$\niii$}{N $\textsc${iii}]}$
$\newcommand{$\heii$}{He $\textsc${ii}}$
$\newcommand{$\civ$}{C $\textsc${iv}}$
$\newcommand{$\niv$}{N $\textsc${iv}}$
$\newcommand{$\todo$}[1]{{\color{blue} \tt #1}}$
$\newcommand{$\tbc$}[1]{#1 ({\color{red} \tt TBC})}$
$\newcommand{$\tbd$}{({\color{red} \tt TBD})}$
$\newcommand{$\nod$}{$\nod$ata}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07234-b31b1b.svg)](https://arxiv.org/abs/2302.07234)<mark>Appeared on: 2023-02-15</mark> - _Submitted to ApJ; 18 pages, 8 figures; comments welcome_

</div>
<div id="authors">

Sandro Tacchella, et al. -- incl., <mark>Jakob M. Helton</mark>, <mark>Hans-Walter Rix</mark>, <mark>Anna de Graaff</mark>, <mark>Lily Whitler</mark>

</div>
<div id="abstract">

**Abstract:** We present JWST NIRCam 9-band near-infrared imaging of the luminous$z=10.6$galaxy GN-z11 from the JWST Advanced Deep Extragalactic Survey (JADES) of the GOODS-N field.  We find a spectral energy distribution (SED) entirely consistent with the expected form of a high-redshift galaxy: a clear blue continuum from 1.5 to 4 microns with a complete dropout in F115W.  The core of GN-z11 is extremely compact in JWST imaging.  We analyze the image with a two-component model, using a point source and a Sérsic profile that fits to a half-light radius of 200 pc and an index$n=0.9$. We find a low-surface brightness haze about$0.4"$to the northeast of the galaxy, which is most likely a foreground object but might be a more extended component of GN-z11. At a spectroscopic redshift of 10.60\citep{bunker23}, the comparison of the NIRCam F410M and F444W images spans the Balmer jump. From population synthesis modeling, here assuming no light from an active galactic nucleus, we reproduce the SED of GN-z11, finding a stellar mass of$\sim10^{9} M_{\odot}$, a star-formation rate of$\sim20 M_{\odot} \mathrm{yr}^{-1}$and a young stellar age of$\sim20$Myr. As massive galaxies at high redshift are likely to be highly clustered, we search for faint neighbors of GN-z11, finding 9 galaxies out to$\sim$5 comoving Mpc transverse with photometric redshifts consistent with$z=10.6$, and a 10$^{\rm th}$more tentative dropout only$3"$away.

</div>

<div id="div_fig1">

<img src="tmp_2302.07234/./ratio_flux_size.png" alt="Fig1" width="100%"/>

**Figure 1. -** Posterior distribution of the fraction of $F200W$ flux in the Extended component as a function of its half-light size.
We obtain this directly from the ForcePho modeling. We find an interesting covariance: the larger the Extended component, the less flux it contributes. Importantly, there is no tail towards zero size or zero flux, emphasizing that the Extended component is clearly detected and indeed extended.  (*fig:flux_ratio_size*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07234/./SED.png" alt="Fig2" width="100%"/>

**Figure 2. -** The spectral energy distribution and thumbnail images of GN-z11 from 9 JWST NIRCam bands. Each thumbnail square is 1.5 arcsec on a side. The blue points show the 7 detected NIRCam bands ("Galaxy" in Table \ref{tab:modelphotom}), while the 2$\sigma$ upper limits for F090W and F115W are indicated as downward pointing arrows. The red solid line and the shaded region shows the median the 16th-84th percentile of the SED posterior from the Prospector modeling (Section \ref{sec:sed}).  (*fig:sed*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07234/./FP_residual.png" alt="Fig4" width="100%"/>

**Figure 4. -** The scene fitting from ForcePho.  We model the immediate region of GN-z11 with 3 components (in addition to the bright source on the left): a point source, an extended Sérsic component near that source, and a second Sérsic centered on the haze. The two elliptical Sérsic components are free in index $n$, half-light radius, position angle, axis ratio, and centroid.  All three components vary freely in amplitude in each band.  A fourth S\'ersic component is used to model the brighter galaxy on the left of the thumbnails.  ForcePho fits to the individual exposures, producing a Markov Chain sampling of the likelihood of the scene. To display these residuals, we subtracted the model from the data exposures and then combined the exposures into a mosaic for each band. The residuals are very weak, indicating that the three component model has explained most of the scene.  We note that these images are displayed in pixel coordinates, unlike other images in this paper, which use the north-up mosaic. All maps are showing flux in nJy (see color bar) in each pixel.
 (*fig:FP_residual*)

</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\niii}{N \textsc{iii}]}$
$\newcommand{\niv}{N \textsc{iv}]}$
$\newcommand{\hei}{He \textsc{i}}$
$\newcommand{\heii}{He \textsc{ii}}$
$\newcommand{\ciii}{C \textsc{iii}]}$
$\newcommand{\oii}{[O \textsc{ii}]}$
$\newcommand{\oiii}{[O \textsc{iii}]}$
$\newcommand{\oiiiSF}{O \textsc{iii}]}$
$\newcommand{\neiii}{[Ne \textsc{iii}]}$
$\newcommand{\niiiL}{N \textsc{iii}] \lambda}$
$\newcommand{\nivL}{N \textsc{iv}] \lambda}$
$\newcommand{\ciiiL}{C \textsc{iii}] \lambda}$
$\newcommand{\ciiiLL}{C \textsc{iii}] \lambda\lambda}$
$\newcommand{\civ}{C \textsc{iv}}$
$\newcommand{\oiiL}{[O \textsc{ii}] \lambda\lambda}$
$\newcommand{\oiiiL}{[O \textsc{iii}] \lambda}$
$\newcommand{\oiiiSFL}{O \textsc{iii}] \lambda\lambda}$
$\newcommand{\neiiiL}{[Ne \textsc{iii}] \lambda}$
$\newcommand{\hd}{H \delta}$
$\newcommand{\hg}{H \gamma}$
$\newcommand{\hb}{H \beta}$
$\newcommand{\mgii}{Mg \textsc{ii}}$
$\newcommand{\oiiiau}{[O \textsc{iii}]}$
$\newcommand{\oiiiauL}{[O \textsc{iii}] \lambda}$
$\newcommand{\xhi}{x_{\rm \textsc{hi}}}$
$\newcommand{\jwst}{\emph{JWST}}$
$\newcommand{\flux}{erg s^{-1} cm^{-2}}$
$\newcommand{\kms}{km s^{-1}\xspace}$
$\newcommand{\asec}{^{\prime\prime}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\lya$}{Ly\alpha}$
$\newcommand{$\niii$}{N $\textsc${iii}]}$
$\newcommand{$\niv$}{N $\textsc${iv}]}$
$\newcommand{$\hei$}{He $\textsc${i}}$
$\newcommand{$\hei$i}{He $\textsc${ii}}$
$\newcommand{$\ciii$}{C $\textsc${iii}]}$
$\newcommand{$\oii$}{[O $\textsc${ii}]}$
$\newcommand{$\oii$i}{[O $\textsc${iii}]}$
$\newcommand{$\oii$iSF}{O $\textsc${iii}]}$
$\newcommand{$\neiii$}{[Ne $\textsc${iii}]}$
$\newcommand{$\niii$L}{N $\textsc${iii}] \lambda}$
$\newcommand{$\niv$L}{N $\textsc${iv}] \lambda}$
$\newcommand{$\ciii$L}{C $\textsc${iii}] \lambda}$
$\newcommand{$\ciii$LL}{C $\textsc${iii}] \lambda\lambda}$
$\newcommand{$\civ$}{C $\textsc${iv}}$
$\newcommand{$\oii$L}{[O $\textsc${ii}] \lambda\lambda}$
$\newcommand{$\oii$iL}{[O $\textsc${iii}] \lambda}$
$\newcommand{$\oii$iSFL}{O $\textsc${iii}] \lambda\lambda}$
$\newcommand{$\neiii$L}{[Ne $\textsc${iii}] \lambda}$
$\newcommand{$\hd$}{H \delta}$
$\newcommand{$\hg$}{H \gamma}$
$\newcommand{$\hb$}{H \beta}$
$\newcommand{$\mgii$}{Mg $\textsc${ii}}$
$\newcommand{$\oii$iau}{[O $\textsc${iii}]}$
$\newcommand{$\oii$iauL}{[O $\textsc${iii}] \lambda}$
$\newcommand{$\xhi$}{x_{\rm $\textsc${hi}}}$
$\newcommand{$\jwst$}{\emph{JWST}}$
$\newcommand{$\flux$}{erg s^{-1} cm^{-2}}$
$\newcommand{$\kms$}{km s^{-1}$\xspace$}$
$\newcommand{$\asec$}{^{\prime\prime}}$</div>



<div id="title">

#  luminous galaxy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07256-b31b1b.svg)](https://arxiv.org/abs/2302.07256)<mark>Appeared on: 2023-02-15</mark> - _Submitted to Astronomy & Astrophysics, 14 pages, 9 figures_

</div>
<div id="authors">

Andrew J. Bunker, et al. -- incl., <mark><mark>Anna de Graaff</mark></mark>, <mark><mark>Hans-Walter Rix</mark></mark>

</div>
<div id="abstract">

**Abstract:** We present JADES JWST/NIRSpec spectroscopy of GN-z11, the most luminous candidate$z>10$Lyman break galaxy in the GOODS-North field with$M_{UV}=-21.5$. We derive a redshift of$z=10.603$(lower than previous determinations) based on multiple emission lines in our low and medium resolution spectra over$0.8-5.3 \mu$m. We significantly detect the continuum and measure a blue rest-UV spectral slope of$\beta=-2.4$. Remarkably, we see spatially-extended Lyman-$\alpha$in emission (despite the highly-neutral IGM expected at this early epoch), offset 555 km s$^{-1}$redward of the systemic redshift. From our measurements of collisionally-excited lines of both low- and high-ionization (including[O$\textsc${ii}]$\lambda3727$,[Ne$\textsc${iii}]$\lambda 3869$and C$\textsc${iii}]$\lambda1909$) we infer a high ionization parameter ($\log U\sim -2$). We detect the rarely-seen N$\textsc${iv}]$\lambda1486$and N$\textsc${iii}]$\lambda1748$lines in both our low and medium resolution spectra, with other high ionization lines seen in low resolution spectrum such as He$\textsc${ii}(blended with O$\textsc${iii}]) and C$\textsc${iv}(with a possible P-Cygni profile). Based on the observed rest-UV line ratios, we cannot conclusively rule out photoionization from AGN. The high C$\textsc${iii}]/He$\textsc${ii}ratios, however, suggest a likely star-formation explanation. If the observed emission lines are powered by star formation, then the strong N$\textsc${iii}]$\lambda1748$observed may imply an unusually high$N/O$abundance. Balmer emission lines (H$\gamma$, H$\delta$) are also detected, and if powered by star formation rather than an AGN we infer a star formation rate of$\sim 20-30 M_{\odot} \rm yr^{-1}$(depending on the IMF) and low dust attenuation. Our NIRSpec spectroscopy confirms that GN-z11 is a remarkable galaxy with extreme properties seen 430 Myr after the Big Bang.

</div>

<div id="div_fig1">

<img src="tmp_2302.07256/./figures/all_emlines_gnz11_v2.png" alt="Fig7" width="100%"/>

**Figure 7. -** Emission lines seen in GN-z11 from the Medium resolution gratings, apart from the last row that shows line emission from the low resolution prism spectrum. The $\civ$, $\hei$i+$\oii$iSF$\mgii$ and $\oii$iauL$4363$ lines are not detected with high significance in the grating spectra. (*fig:emlines*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07256/./figures/GN-z11_v3GTO_1D_2D_micron_3pix.png" alt="Fig6" width="100%"/>

**Figure 6. -** 2D (top) and 1D (bottom) spectra of GN-z11 using PRISM/CLEAR configuration of NIRSpec. Prominent emission lines present in the spectra are marked. The signal to noise ratio (SNR) of the continuum is high and the emission lines are clearly seen in both the 1D and 2D spectra. (*fig:spectrum*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07256/./figures/GN-z11_Lya_zoom_hist.png" alt="Fig3" width="100%"/>

**Figure 3. -** Zoom in on the $\lya$ emission line in the G140M 1D (lower) and 2D (upper) spectra. The grey dashed line shows the systemic wavelength of the $\lya$ transition. The histogram (top-left) of the $\lya$ spatial profile (yellow) and that of the continuum (blue), shows the $\lya$ emission from GN-z11 is more extended towards the south-west (up in the MSA shutter in this view). (*fig:Lya_zoom*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

204  publications in the last 7 days.
	 _build/html/2302.07256.md
	 _build/html/2302.07234.md
	 _build/html/2302.07057.md
	 _build/html/2302.05694.md
	 _build/html/2302.05465.md
	 _build/html/2302.04507.md
	 _build/html/2302.04239.md
	 _build/html/2302.04138.md
	 _build/html/2302.03699.md
	 _build/html/2302.03576.md
	 _build/html/2302.03042.md
	 _build/html/2302.03039.md
	 _build/html/2302.02890.md
	 _build/html/2302.02611.md
	 _build/html/2302.02429.md
	 _build/html/2302.01678.md
	 _build/html/2302.00450.md
	 _build/html/2301.13766.md
	 _build/html/2301.13526.md
	 _build/html/2301.12890.md
	 _build/html/2301.11942.md
	 _build/html/2301.10717.md
	 _build/html/2301.10666.md
	 _build/html/2301.10496.md
	 _build/html/2301.09621.md
	 _build/html/2301.09140.md
	 _build/html/2301.08770.md
	 _build/html/2301.08310.md
	 _build/html/2301.07990.md
	 _build/html/2301.07366.md
	 _build/html/2301.07199.md
	 _build/html/2301.07116.md
	 _build/html/2301.06873.md
	 _build/html/2301.06819.md
	 _build/h

In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

12  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers